Данный ноутбук посвящен визуализации данных для построения микросервиса рекомендаций. Для того чтобы тестировать сервис было проще, ниже выведены случайные пользователи и треки, содержащиеся в файлах с персональными рекомендациями и схожими треками.

Это облегчит поиск пользователей и треков для использования в тестировании, ведь в процессе создания рекомендаций на предыдущем этапе не все пользователи получили персональные рекомендации, а также не все треки попали в таблицу схожести.

In [2]:
import json

import numpy as np
import pandas as pd

RANDOM_SEED = 42

## Inspecting users with personal recommendations

In [3]:
recs = pd.read_parquet("data/recommendations.parquet")
recs

,user_id,track_id,als_score,genre,genre_share,count,cb_score,rank
3,4,81848875,3.347138e-20,pop,0.245902,11436.0,0.832675,1
4,4,60292250,3.272448e-20,rnb,0.008675,16624.0,0.818144,2
9,4,70619465,2.749917e-20,pop,0.245902,10130.0,0.726640,3
7,4,78194999,2.905115e-20,rusrap,0.112467,7192.0,0.671969,4
8,4,67538121,2.766610e-20,rusrap,0.112467,6769.0,0.493854,5
...,...,...,...,...,...,...,...,...
1339771,1374567,39257277,1.553284e-19,dance,0.067932,17035.0,0.421597,6
1339775,1374567,39307896,1.224216e-19,rusrap,0.112467,11005.0,0.413653,7
1339779,1374567,18820599,9.189979e-20,rock,0.047187,9775.0,0.358984,8
1339777,1374567,81275802,9.526836e-20,pop,0.245902,3952.0,0.175735,9


In [4]:
np.random.seed(RANDOM_SEED)
ids = recs.user_id.unique()
print("30 random users with personal recommendations:\n")
np.random.choice(ids, 30)

30 random users with personal recommendations:



array([1252227, 1353551, 1064970, 1231477, 1132287,  562737,  898002,
       1157822, 1296326,  163687,  421798,  689442,  664828,    8105,
        613396,  665746,   54633, 1061568,  875493, 1340732,  867124,
       1257632, 1339850,   28073, 1271189,  674253, 1333963,  868799,
       1225930,  691737], dtype=int32)

## Inspecting tracks with similar ones

In [5]:
similar_tracks = pd.read_parquet("data/similar.parquet")
similar_tracks

,track_id_1,track_id_2,score
0,26,139,0.000000
1,26,138,0.000000
2,26,136,0.000000
3,26,135,0.000000
4,26,38,0.000000
...,...,...,...
299995,446684,53726952,0.757322
299996,446684,29028731,0.755143
299997,446684,39178075,0.734521
299998,446684,38184093,0.722108


In [6]:
np.random.seed(RANDOM_SEED)
ids = similar_tracks.track_id_1.unique()
print("30 random tracks with similar tracks:\n")
np.random.choice(ids, 30)

30 random tracks with similar tracks:



array([ 97161,   3911, 315672, 373807,  69522,  35926, 105355, 306718,
       126191, 409598, 367482,  99045, 342136,   8449,   3593,  12309,
        29411, 312370, 325573, 106999, 113305, 207055,  36368, 184351,
       345969, 168561, 110218, 109123,  14198,   1168], dtype=int32)

## Displaying recommendations retrieved

Ячейки ниже имеет смысл запускать при наличии перемешанных рекомендаций в файле `recs_blended.json`, который был получен после серии тестовых запросов к микросервису при помощи скрипта `test_service.py`. 

Ниже приводятся наглядно, какие треки рекомендуются нашей системой пользователю 54633:

In [7]:
items = pd.read_parquet("data/items.parquet")
items

,track_id,track,artist,album,genre,track_id_encoded
0,26,Complimentary Me,Elizabeth & the Catapult,Taller Children,pop,0
1,38,Momma's Boy,Elizabeth & the Catapult,Taller Children,pop,1
2,135,Atticus,Noisettes,Wild Young Hearts,pop,2
3,136,24 Hours,Noisettes,Wild Young Hearts,pop,3
4,138,Don't Upset The Rhythm (Go Baby Go),Noisettes,Wild Young Hearts,pop,4
...,...,...,...,...,...,...
999995,101478482,На лицо,FLESH,На лицо,rusrap,999995
999996,101490148,Без капли мысли,Даня Милохин,Без капли мысли,pop,999996
999997,101493057,SKITTLES,WhyBaby?,SKITTLES,foreignrap,999997
999998,101495927,Москва,Yanix,Москва,rusrap,999998


In [8]:
with open("recs_blended.json", "r") as file:
    recs_blended = json.load(file)
recs_blended = recs_blended["recs"]
recs_blended

[65760936, 35505245, 24321869, 38572747, 6696282]

In [9]:
items_selected = items.query("track_id in @recs_blended")
items_selected.set_index("track_id").reindex(recs_blended).reset_index()

,track_id,track,artist,album,genre,track_id_encoded
0,65760936,Не забывай меня,Anthony El Mejor,Не забывай меня,pop,828658
1,35505245,I Got Love,Miyagi & Эндшпиль,I Got Love,rusrap,512157
2,24321869,Другие берега,Ассаи,Reunion,rusrap,362367
3,38572747,Люби меня люби,гречка,Звёзды только ночью,local-indie,561270
4,6696282,I'm Alive,Paul Oakenfold,Vexille Soundtrack,alternative,196946
